In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go

In [8]:
data = pd.read_csv('../dataset/UQ Tesla Battery - 2022 (1-min resolution).csv')
data.columns

Index(['Measurement time', 'FCAS Event (%)', 'Full Charge Energy (kWh)',
       'Nominal Energy (kWh)', 'Expected Energy (kWh)', 'Charge P Max (kW)',
       'Discharge P Max (kW)', 'Available Blocks', '3 Phase Voltage (V)',
       '3 Phase Current (A)', '3 Phase Power (kW)',
       '3 Phase Reactive Power (kVAr)', '3 Phase Apparent Power (kVA)',
       'Power Factor', 'Frequency (Hz)', 'Real Energy Imported (kWh)',
       'Real Energy Exported (kWh)', 'Reactive Energy Imported (kVArh)',
       'Reactive Energy Exported (kVArh)', 'Apparent Energy (kVAh)',
       'Energy Price ($/MWh)', 'Raise 6sec Price ($/MWh)',
       'Raise 60sec Price ($/MWh)', 'Raise 5min Price ($/MWh)'],
      dtype='object')

In [136]:
fig = go.Figure()
i = np.random.randint(40,50)
fig.add_trace(go.Scatter(y=train_dr[i],name='Charging'))
fig.add_trace(go.Scatter(y=train_price[i],yaxis='y2',name='price'))
#fig.add_trace(go.Scatter(y=data[288*i:288*(i+1)]['Frequency (Hz)'],yaxis='y2',name='frequency'))
fig.update_layout(
    yaxis1={'title':'Chargin Power','range':[-1.5,1.5]},
    yaxis2={'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right','range':[0,200], 'showgrid':False},
)
fig.show()

In [138]:
data = pd.read_csv('../dataset/UQ Tesla Battery - 2022 (5-min resolution).csv')
df_price = data['Energy Price ($/MWh)']
df_y = data['3 Phase Power (kW)']

T = 48 # one day
N_train = 40
N_test = 10
model = MLP(T,T)
train_price = np.mean(df_price[:N_train*T].values.reshape(N_train,T, -1), axis=2)
train_dr = np.mean(df_y[:N_train*T].values.reshape(N_train,T, -1), axis=2)/1000
test_price = np.mean(df_price[N_train*T:(N_test+N_train)*T].values.reshape(N_test,T, -1), axis=2)
test_dr = np.mean(df_y[N_train*T:(N_test+N_train)*T].values.reshape(N_test,T, -1), axis=2)/1000

history = model.fit(train_price,train_dr, validation_data = (test_price, test_dr),  epochs=1000, batch_size=32, verbose=0) #validation_data = (test_price, test_dr),
L = history.history['loss']
val_L = history.history['val_loss']

In [140]:
model2 = RNNmodel(T,T)
history2 = model2.fit(train_price,train_dr, validation_data = (test_price, test_dr),  epochs=1000, batch_size=32, verbose=0) #validation_data = (test_price, test_dr),
L2 = history.history['loss']
val_L2 = history.history['val_loss']

In [104]:
RNN_predict = model2.predict(test_price)
MLP_predict = model.predict(test_price)

In [139]:
val_L[-5:]

[0.2678595781326294,
 0.2678561806678772,
 0.2678542733192444,
 0.26785290241241455,
 0.2678532898426056]

In [119]:
fig = go.Figure()
i = np.random.randint(10)
fig.add_trace(go.Scatter(y=test_dr[i]*1000,name='True'))
#fig.add_trace(go.Scatter(y=test_price[i],yaxis='y2',name='price'))
fig.add_trace(go.Scatter(y=MLP_predict[i]*1000,name='MLP'))
fig.add_trace(go.Scatter(y=RNN_predict[i]*1000,name='RNN'))
#fig.add_trace(go.Scatter(y=data[288*i:288*(i+1)]['Frequency (Hz)'],yaxis='y2',name='frequency'))
fig.update_layout(
    yaxis1={'title':'Chargin Power','range':[-1200,1200]},
    yaxis2={'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right','range':[0,200], 'showgrid':False},
)
fig.show()

In [29]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
import tensorflow as tf
def RNNmodel(input_dim, output_dim):
    model = Sequential()
    model.add(LSTM(units=output_dim,return_sequences=True,input_shape=(input_dim, 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=output_dim,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=output_dim,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=output_dim))
    model.compile(optimizer='adam',loss='mean_squared_error')
    return model

def MLP(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim))
    model.compile(optimizer='adam',loss='mean_squared_error')
    return model